In [33]:
import json

In [67]:
with open("data/raw_data.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [109]:
data=[]
for user in jsonObject:
    for message in user['turns'][::2]:
        sub = {}
        sub['text']= message['text']
        sub['intent'] = 'BookFlight'
        
        entities_name =  json_extract(message["labels"]['acts'], 'val')
        entities_value =  json_extract(message["labels"]['acts'], 'key')
        # print(entities_value)
        indexes=[]
        indexes = [i for i in range(len(entities_name)) if not isinstance(entities_name[i],str)]
        if indexes != []:
            if len(indexes)>1:
                indexes.sort(reverse=True)
            for index in indexes:
                entities_name.pop(index)
                entities_value.pop(index)
        positions=[]    
        if len(entities_name)>0:
            positions = [(message['text'].find(substring),message['text'].find(substring) + len(substring) - 1 ) for substring in entities_name]
        ind=[]
        ind= [i for i in range(len(positions)) if positions[i][0] == -1]
        if len(ind)>0:
            if len(ind)>1:
                ind.sort(reverse=True)
            for index in ind:
                entities_name.pop(index)
                entities_value.pop(index)
                positions.pop(index)    
        entities_filters = ['or_city', 'price', 'dst_city', 'dep_time_dst', 'dep_time_or', 'budget', 'intent','duration','action', 'str_date', 'end_date']
                
        entities = []
        if len(entities_name)>0:
            for entitie_value, position, entitie_name  in zip(entities_value, positions, entities_name):
                # if  any([ent in entities_filters for ent in entitie_value] ):
                if entitie_value in entities_filters:
                    entitie = {"entity": entitie_value,
                            "startPos":position[0],
                            "endPos":position[1]}
                    entities.append(entitie)
                
        sub['entities'] = entities
        if entities != []:
            data.append(sub)
with open('utterances.json', 'w', encoding='utf8') as f:
    json.dump(data, f,ensure_ascii=False)

In [95]:
entities_list = list(set(json_extract(data,'entity')))
entities_json = []
for entity in entities_list:
    substr = {"name":entity,
              "children":[],
              "roles":[],
              "features":[]}
    entities_json.append(substr)
with open('entities.json', 'w') as f:
    json.dump(entities_json, f)


In [37]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [104]:
array = [1, 2, 3]
az = [5,4]
exists = any( [a in array for a in az])

print(exists)

False
